# Vibrational Modes of Iso-Propanol
Due to availability modes were calculated with a software used for quantum-chemistry.

In [1]:
import psi4
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem

In [2]:
%config Completer.use_jedi = False

In [ ]:
molecule = Chem.MolFromSmiles('CC(O)C')
molecule = Chem.AddHs(molecule)

# Optimzie geometry with rdkit
AllChem.EmbedMolecule(molecule)
# Transforming mol Block to a format Psi4 understands
mol_block = Chem.MolToXYZBlock(molecule)[4:-1]
mol_block = "\n".join(["0 1", mol_block, "units angstrom"])

In [7]:
# Setting psi4 parameter
psi4.set_memory('5000 MB')
psi4.set_options({"PARALLEL": True,
                 'reference': 'rhf'})
# Creating a psi4 molecule
iso_prop = psi4.geometry(mol_block)

# Evaluating it's energy
psi4.energy('scf/cc-pvdz')

-193.11960581545182

In [8]:
# Optimizing Molecule geometry
psi4.optimize('scf/cc-pvdz', molecule=iso_prop)

Optimizer: Optimization complete!


-193.13501268787556

In [10]:
# Frequency calculation. May take time...
# scf: self-consistent field a.k.a. Hartree–Fock method
# cc-pvdz: correlation-consistent basis sets (https://aip.scitation.org/doi/10.1063/1.456153)
#          
scf_e, scf_wfn = psi4.frequency('scf/cc-pvdz', molecule=iso_prop, return_wfn=True)

In [11]:
# https://github.com/psi4/psi4/blob/3d2403c43e2cf8df699ff6e382a4ae2e3a0a5faa/psi4/driver/qcdb/vib.py#L177
# Modified this funtion to return a format readable by Avogrado

def wfn2molen_vibrational_modes(wfn):
    mol = wfn.molecule()
    geom = np.asarray(mol.geometry())
    atom_symbol = [mol.symbol(at) for at in range(mol.natom())]
    vibinfo = wfn.frequency_analysis
    """Format vibrational analysis for Molden.
    Parameters
    ----------
    vibinfo : dict of vibration Datum
        Holds results of vibrational analysis.
    atom_symbol : array-like of str
        (nat,) element symbols for geometry of vibrational analysis.
    geom : array-like of float
        (nat, 3) geometry of vibrational analysis [a0].
    standalone : bool, optional
        Whether returned string prefixed "[Molden Format]" for standalone rather than append.
    Returns
    -------
    str
        `vibinfo` formatted for Molden, including FREQ, FR-COORD, & FR-NORM-COORD fields.
    Notes
    -----
    Molden format spec from http://www.cmbi.ru.nl/molden/molden_format.html
    Specifies "atomic coordinates x,y,z and atomic displacements dx,dy,dz are all in Bohr (Atomic Unit of length)"
    Despite it being quite wrong, imaginary modes are represented by a negative frequency.
    
    Modified to be accepted by Avogrado. 
    """
    nat = int(len(vibinfo['q'].data[:, 0]) / 3)
    active = [idx for idx, trv in enumerate(vibinfo['TRV'].data) if trv == 'V']

    text = '[Molden Format]\n'

    text += """[FREQ]\n"""
    for vib in active:
        if vibinfo['omega'].data[vib].imag > vibinfo['omega'].data[vib].real:
            freq = -1.0 * vibinfo['omega'].data[vib].imag
        else:
            freq = vibinfo['omega'].data[vib].real
        text += """{:11.4f}\n""".format(freq)

    text += """[FR-COORD]\n"""
    for at in range(nat):
        text += ("{:>2}{:13.6f}{:13.6f}{:13.6f}\n").format(atom_symbol[at], *geom[at])

    text += """[FR-NORM-COORD]\n"""
    for idx, vib in enumerate(active):
        text += """vibration {}\n""".format(idx + 1)
        for at in range(nat):
            text += ('   ' + """{:11.6f}""" * 3 + '\n').format(*(vibinfo['x'].data[:, vib].reshape(nat, 3)[at].real))


    return text

In [14]:
# Writing result to ouput file
with open("iso_prop.molden", "w") as out:
    out.write(wfn2molen_vibrational_modes(scf_wfn))